In [72]:
import requests as req
import polars as pl
import json
import datetime

from scipy.optimize import minimize
pl.Config.set_tbl_rows(30)

polars.config.Config

In [74]:
class Trading212Client:
    def __init__(self, api_key = "23258933ZjHilkBcSdxjMQxlvDdBmNmWUnpyz", base_url="https://live.trading212.com"):
        self.base_url = base_url
        self.headers = {
            "Authorization": api_key,
            "Content-Type": "application/json"
        }
    
    def get_pies(self):
        """Fetch all pies from the account"""
        endpoint = "/api/v0/equity/pies"
        response = req.get(f"{self.base_url}{endpoint}", headers=self.headers)
        return pl.DataFrame(response.json())
    
    def update_pie(self, pie_id, pie_request):
        """Update a pie with new allocations"""
        endpoint = f"/api/v0/equity/pies/{pie_id}"
        response = req.post(
            f"{self.base_url}{endpoint}", 
            headers=self.headers,
            json=pie_request
        )
        return pl.DataFrame(response.json())
    
    def get_instruments(self):
        """Fetch all available instruments"""
        endpoint = "/api/v0/equity/metadata/instruments"
        response = req.get(f"{self.base_url}{endpoint}", headers=self.headers)
        return pl.DataFrame(response.json())
    
    def get_portfolio(self):
        """Fetch current portfolio positions"""
        endpoint = "/api/v0/equity/portfolio"
        response = req.get(f"{self.base_url}{endpoint}", headers=self.headers)
        return pl.DataFrame(response.json())

In [67]:
positions_df = pl.read_csv("O_Positions.csv")
instruments_df = pl.read_csv("all_instruments.csv")

# Join Dataset to get full position informations
portfolio_df = positions_df.join(
    instruments_df.select(["ticker", "type", "name", "currencyCode"]),
    on = "ticker",
    how = "left"
)
portfolio_df = portfolio_df.with_columns([
    pl.col("initialFillDate").str.strptime(pl.Datetime)
])


In [71]:
# Calculate Portfolio Metrics
total_value = (portfolio_df["quantity"] * portfolio_df["currentPrice"]).sum()
position_weights = (positions_df["quantity"] * positions_df["currentPrice"]) / total_value

exposure_df = portfolio_df.group_by("type").agg([
    (pl.col("quantity") * pl.col("currentPrice")).sum().alias("exposure_value"),
    ((pl.col("quantity") * pl.col("currentPrice")) / total_value).sum().alias("exposure_pct")
]).sort("exposure_pct", descending = True)

exposure_df

type,exposure_value,exposure_pct
str,f64,f64
"""ETF""",118056.119445,0.917955
"""STOCK""",10551.625763,0.082045
